### **Fact Orders**

**Data Reading**

In [0]:
df = spark.sql("select * from databricks_catalog.silver.orders_silver")
display(df)

In [0]:
df_dimcus = spark.sql("select DimCustomerKey, customer_id as dim_customer_id from databricks_catalog.gold.dimcustomers")
df_dimpro = spark.sql("select product_id as DimProductKey, product_id as dim_product_id from databricks_catalog.gold.dimproducts")
display(df_dimpro)


In [0]:
df_fact = df.join(df_dimcus, df["customer_id"] == df_dimcus["dim_customer_id"], how="left")\
            .join(df_dimpro, df["product_id"] == df_dimpro["dim_product_id"], how = "left")
df_fact_new = df_fact.drop("dim_customer_id", "dim_product_id", "customer_id", "product_id")
df_fact_new.display()


**Upsert on Fact Table**

In [0]:
  from delta.tables import DeltaTable
  if spark.catalog.tableExists("databricks_catalog.gold.FactOrders"):
      
      dlt_obj = DeltaTable.forName(spark, "databricks_catalog.gold.FactOrders")
      dlt_obj.alias("trg").merge(df_fact_new.alias("src"), "trg.order_id = src.order_id AND trg.DimCustomerKey = src.DimCustomerKey AND trg.DimProductKey = src.DimProductKey")\
          .whenMatchedUpdateAll()\
          .whenNotMatchedInsertAll()\
          .execute()


  else:
      df_fact_new.write.format("delta")\
          .option("path", "abfss://gold@etedatabricks01.dfs.core.windows.net/FactOrders")\
          .saveAsTable("databricks_catalog.gold.FactOrders")

In [0]:
%sql
use catalog `databricks_catalog`; select * from `gold`.`factorders` ;